In [1]:
from nltk.corpus import stopwords
import os
import re
import nltk 
import numpy as np
from nltk.stem.wordnet import WordNetLemmatizer
import sklearn
import pandas as pd
import time
import math

In [2]:
def get_filenames(rootDir):
# This function returns the filenames and categories of articles in the given directory

# categories will store the names of all type of articles
# documents_count stores the no. of documents in a particular category
# docslist stores the list of all documents
    categories =[]
    docslist = []
    dirnamelist = []

    for dirName, subdirList, fileList in os.walk(rootDir):

        if len(subdirList) >0:
            categories = subdirList

        if len(subdirList) == 0:
            docslist.append(fileList)
            dirnamelist.append(dirName)

    return categories,docslist

In [3]:
def get_clean_data(docslist,rootDir,categories):
# This function reads all the documents in the given directory and cleans the text data and generates tokens
# vocab_list stores the vocabulary of our dtaset    
# cateogory_docs stores the words per document per category
    vocab_list = []
    category_docs = []

    for index,dlist in enumerate(docslist):
        
        #docs_words stores the filtered words of documents in a category
        
        docs_words = []
        for doc in dlist:
            print('\n Reading document',doc)
            
            f=open('{0}/{1}/{2}'.format(rootDir,categories[index],doc),'r')

            wlist = f.read()
            
            #using regular expression to remove special and numeric characters from text document
            wlist = re.sub('[^A-Za-z]+', ' ', wlist)
            wlist = re.sub(r'\b\w{1,2}\b', '', wlist)
            wlist = re.sub(r'\w*\d\w*', '', wlist).strip()
            word_list = re.findall(r"[\w']+", wlist)
            tokens = [token.lower() for token in word_list]
            
            #removing stopwords from the tokens
            filtered_words = [word for word in tokens if word not in stopwords.words('english')]
            
            #removing words with length less than 3.
            for lword in filtered_words:
                if len(lword) < 3:
                    filtered_words.remove(lword)

            
            
            #appending unique words from the filtered list to our vocabulary list
            for uword in filtered_words:
                if uword not in vocab_list:
                    vocab_list.append(uword)

            docs_words.append(filtered_words)
        category_docs.append(docs_words)
        

    return vocab_list,category_docs



In [4]:
def frequency_dataset(vocab_list,categories,category_docs):
    # this functions returns a dataframe with frequency of each word in the vocabulary in all the documents
    # data stores the frequency data
    # target stores the category of each document
    # feature_frequency is a dictionary which stores the total frequency of every word in the vocabulary
    data=[]
    target=[]
    feature_frequency={}
    count=0
    for category_index in range(0,len(category_docs)):
        
        
        for doc in range(0,len(category_docs[category_index])):
            target.append(categories[category_index])
            data.append(list(np.zeros(len(vocab_list),int)))
            
            for word in category_docs[category_index][doc]:
                
                index=vocab_list.index(word)
                data[count][index]+=1
                if word in feature_frequency.keys():
                    feature_frequency[word]+=1
                else:
                    feature_frequency[word]=1
            count+=1
    data_df=pd.DataFrame(data)
    data_df.columns=vocab_list
    return data_df,target,feature_frequency  

In [5]:
def eliminate_features(data_df,feature_frequency,vocab_list):
    #this function eliminates words and their frequency data from the vocabulary_list and data_df with small overall frequencies
    # and takes top 1000 words as final dataset
    new=sorted(feature_frequency.items(), key=lambda t:t[1], reverse=True)
    feature_frequency=new[:1000]
    final_data=pd.DataFrame()
    final_vocab=[]
    for i in dict(feature_frequency).keys():
        final_data[i]=data_df[i]
        final_vocab.append(i)
    return final_data,final_vocab
    

In [6]:
def fit(X_train,Y_train):#data to be passed as dataframe
    # this function returns a dictionary with category-wise the overall frequency of each word in vocabulary
    result={}
    classes=set(Y_train[0])
    for class_ in classes:
        result[class_]={}
        result["total-docs"]=len(Y_train[0])
        for feature in X_train.columns:
            result[class_][feature]=X_train[feature][Y_train[0]==class_].sum()
        result[class_]["total_words"]=sum(result[class_].values())
        result[class_]["class_frequency"]=len(Y_train[Y_train[0]==class_])
    return result

In [7]:
def probability(dictionary, x, current_class):
    output=np.log(dictionary[current_class]["class_frequency"])-np.log(dictionary["total-docs"])
    
    num_features = len(list(dictionary[current_class].keys())[:1000])
    for j in range(0, num_features):
        xj = x[j]
        if xj!=0:
            count_current_class_with_value_xj = dictionary[current_class][list((dictionary[current_class].keys()))[j]] + 1
            count_current_class = dictionary[current_class]["total_words"] + len(dictionary[current_class].keys())
            current_xj_probablity = np.log(count_current_class_with_value_xj) - np.log(count_current_class)
            output = output + current_xj_probablity
    return output

In [8]:
def predictSinglePoint(dictionary, x):
    #this function predicts the category of a single document
    #best_prob is the maximum prbability out of all the probabilities of the classes to be the category of the document.
    #best_class is the best prediction for the document(decided on the basis of best_probability)
    
    classes = list(dictionary.keys())
    classes.remove('total-docs')
    best_prob = -math.inf
    best_class = -1
    first_run = True
    for current_class in classes:
        #p_current_class stores the probability of current class
        p_current_class = probability(dictionary, x, current_class)
        
        if (first_run or p_current_class > best_prob):
            best_prob = p_current_class
            best_class = current_class
        first_run = False
    return best_class

In [9]:
def predict(dictionary,X_test):
    #this functions predicts the category of the documents passed in the dataset
    y_pred=[]
    x_t=np.array(X_test)
    i=0
    for x in x_t:
        x_class=predictSinglePoint(dictionary, x)
        y_pred.append(x_class)
        i+=1
    return y_pred

In [10]:
start=time.time()
categories ,docslist=get_filenames(r"C:\Users\nEW u\ML_A\Machine Learning\Naive Bayes\Text Classification\mini_newsgroups")
end=time.time()
print("time taken:  ",end-start)

time taken:   0.050865888595581055


In [11]:
start=time.time()
vocab_list,category_docs=get_clean_data(docslist,r"C:\Users\nEW u\ML_A\Machine Learning\Naive Bayes\Text Classification\mini_newsgroups",categories)
end=time.time()
print("time taken:  ",end-start)


 Reading document 51121

 Reading document 51126

 Reading document 51127

 Reading document 51131

 Reading document 51139

 Reading document 51143

 Reading document 51146

 Reading document 51166

 Reading document 51170

 Reading document 51174

 Reading document 51186

 Reading document 51191

 Reading document 51195

 Reading document 51199

 Reading document 51203

 Reading document 51222

 Reading document 51223

 Reading document 51227

 Reading document 51251

 Reading document 51281

 Reading document 51305

 Reading document 51310

 Reading document 51313

 Reading document 51314

 Reading document 53061

 Reading document 53062

 Reading document 53123

 Reading document 53126

 Reading document 53150

 Reading document 53154

 Reading document 53188

 Reading document 53190

 Reading document 53211

 Reading document 53212

 Reading document 53222

 Reading document 53235

 Reading document 53249

 Reading document 53284

 Reading document 53291

 Reading document 53299



 Reading document 60439

 Reading document 60440

 Reading document 60453

 Reading document 60457

 Reading document 60474

 Reading document 60475

 Reading document 60481

 Reading document 60509

 Reading document 60514

 Reading document 60526

 Reading document 60543

 Reading document 60548

 Reading document 60551

 Reading document 60652

 Reading document 60656

 Reading document 60663

 Reading document 60684

 Reading document 60685

 Reading document 60691

 Reading document 60694

 Reading document 60695

 Reading document 60698

 Reading document 60699

 Reading document 60722

 Reading document 60724

 Reading document 60732

 Reading document 60735

 Reading document 60749

 Reading document 60766

 Reading document 60769

 Reading document 60828

 Reading document 60837

 Reading document 60838

 Reading document 60841

 Reading document 60842

 Reading document 60859

 Reading document 60882

 Reading document 60912

 Reading document 60928

 Reading document 60934



 Reading document 76333

 Reading document 76348

 Reading document 76353

 Reading document 76358

 Reading document 76378

 Reading document 76406

 Reading document 76417

 Reading document 76419

 Reading document 76420

 Reading document 76431

 Reading document 76441

 Reading document 76460

 Reading document 76478

 Reading document 76483

 Reading document 76485

 Reading document 76499

 Reading document 76505

 Reading document 76514

 Reading document 76555

 Reading document 76589

 Reading document 76592

 Reading document 76594

 Reading document 76607

 Reading document 76649

 Reading document 76704

 Reading document 76782

 Reading document 76785

 Reading document 76795

 Reading document 76814

 Reading document 76831

 Reading document 76847

 Reading document 76851

 Reading document 76879

 Reading document 76880

 Reading document 76927

 Reading document 76936

 Reading document 76937

 Reading document 76940

 Reading document 76945

 Reading document 101564


 Reading document 104927

 Reading document 104942

 Reading document 104957

 Reading document 104963

 Reading document 104966

 Reading document 104972

 Reading document 104973

 Reading document 104977

 Reading document 104984

 Reading document 105034

 Reading document 105048

 Reading document 105070

 Reading document 105075

 Reading document 105093

 Reading document 105102

 Reading document 105104

 Reading document 105111

 Reading document 105114

 Reading document 105122

 Reading document 105123

 Reading document 105163

 Reading document 52550

 Reading document 52555

 Reading document 52561

 Reading document 52569

 Reading document 52587

 Reading document 52589

 Reading document 52599

 Reading document 52600

 Reading document 52616

 Reading document 52618

 Reading document 52624

 Reading document 52628

 Reading document 52648

 Reading document 52658

 Reading document 52661

 Reading document 52667

 Reading document 52669

 Reading document 53540

 Re


 Reading document 58786

 Reading document 58791

 Reading document 58807

 Reading document 58811

 Reading document 58826

 Reading document 58840

 Reading document 58842

 Reading document 58852

 Reading document 58863

 Reading document 58867

 Reading document 58875

 Reading document 58893

 Reading document 58903

 Reading document 58915

 Reading document 58931

 Reading document 58947

 Reading document 58951

 Reading document 58952

 Reading document 58961

 Reading document 58988

 Reading document 58994

 Reading document 59001

 Reading document 59002

 Reading document 59004

 Reading document 59012

 Reading document 59026

 Reading document 59031

 Reading document 59044

 Reading document 59058

 Reading document 59076

 Reading document 59078

 Reading document 59092

 Reading document 59093

 Reading document 59111

 Reading document 59121

 Reading document 59133

 Reading document 59154

 Reading document 59161

 Reading document 59165

 Reading document 59167



 Reading document 54404

 Reading document 54416

 Reading document 54417

 Reading document 54429

 Reading document 54433

 Reading document 54446

 Reading document 54447

 Reading document 54449

 Reading document 54450

 Reading document 54452

 Reading document 54469

 Reading document 54479

 Reading document 54518

 Reading document 54535

 Reading document 54538

 Reading document 54560

 Reading document 54570

 Reading document 54586

 Reading document 54590

 Reading document 54591

 Reading document 54592

 Reading document 54611

 Reading document 54616

 Reading document 54624

 Reading document 54626

 Reading document 54630

 Reading document 54633

 Reading document 54634

 Reading document 54637

 Reading document 54659

 Reading document 54660

 Reading document 54675

 Reading document 54697

 Reading document 54698

 Reading document 54714

 Reading document 54715

 Reading document 54726

 Reading document 54728

 Reading document 54748

 Reading document 54843



 Reading document 83919

 Reading document 83977

 Reading document 83979

 Reading document 83981

 Reading document 83983

 Reading document 84053

 Reading document 84060

 Reading document 84068

 Reading document 84083

 Reading document 84103

 Reading document 84115

 Reading document 84120

 Reading document 84152

 Reading document 84164

 Reading document 84178

 Reading document 84194

 Reading document 84195

 Reading document 84212

 Reading document 84244

 Reading document 84251

 Reading document 84255

 Reading document 84278

 Reading document 84290

 Reading document 84293

 Reading document 84302

 Reading document 84309

 Reading document 84316

 Reading document 84317

 Reading document 84324

 Reading document 84345

 Reading document 84350

 Reading document 84351

 Reading document 84355

 Reading document 84356

 Reading document 84359

 Reading document 84398

 Reading document 84400

 Reading document 84413

 Reading document 84436

 Reading document 84510


In [12]:
start=time.time()
data_df,target,feature_frequency=frequency_dataset(vocab_list,categories,category_docs)
end=time.time()
print("time taken:  ",end-start)

time taken:   590.0948526859283


In [13]:
start=time.time()
final_data,final_vocab=eliminate_features(data_df,feature_frequency,vocab_list)
end=time.time()
print("time taken:  ",end-start)

time taken:   1.2885549068450928


In [14]:
from sklearn import model_selection
X_train,X_test,Y_train,Y_test = model_selection.train_test_split(final_data,pd.DataFrame(target),test_size=0.2,random_state=0)

In [15]:
from sklearn.naive_bayes import MultinomialNB

In [16]:
#Predicting the test data using sklearn Multinomial Naive Base
clf=MultinomialNB()
clf.fit(X_train,Y_train[0])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [17]:
Y_pred=clf.predict(X_test)

In [18]:
#calculating the score
from sklearn.metrics import accuracy_score
accuracy_score(Y_test,Y_pred)

0.7725

In [19]:
#fiting the dataset on the implemented Multinomial Naive Baeyes function
start=time.time()
dictionary=fit(X_train,Y_train)
end=time.time()
print("time taken: ",end-start)

time taken:  23.88411283493042


In [20]:
#predicting for X_test
start=time.time()
y_pred=predict(dictionary,X_test)
end=time.time()
print("time taken: ",end-start)

time taken:  16.648385763168335


In [ ]:
#score for prediction by implemented MultinomialNB
accuracy_score(Y_test,y_pred)

# COMPARISON:
The accuracy score of the in-built sklearn algorithm and my implementation is exactly the same, i.e, 0.7725